In [1]:
import itertools
import more_itertools
import math as m
import os 
import networkx as nx

In [2]:
def read_input(graphfile,number_subpath):
    
    trip_data = open(graphfile,'r').read().split('\n')
    i = 0
    listOfGraphs = {}
    k = 0
    
    while(True):
        if "#" in trip_data[i]:
            i = i+1
            N = int(trip_data[i])
            edges = list()
            subpaths = {}
            while(True):
                i = i+1;
                if "#" in trip_data[i]:
                    break;
                if "" == trip_data[i]:
                    break;
                if "subpaths" in trip_data[i]:
                    for j in range(0,number_subpath):
                        i = i + 1
                        line = trip_data[i].split(" ")
                        subpaths[j] = (line[0:len(line)-1])
                    i = i + 4
                if i >= len(trip_data)-1:
                    break;
                line = trip_data[i].split(" ")
                edges.append((line[0],line[1],line[2]))
            G = {'Nodes':N,'list of edges': edges,'subpaths':subpaths}
            print(subpaths)
            listOfGraphs[k] = G
            k +=1 
        if i >= len(trip_data)-1:
                    break; 

    return listOfGraphs;

In [3]:
def flowMultipleDecomposition(data,K):
    
   
    # calculate the minimal flow decomposition based on such graph
    V = data['vertices']
    E = data['edges']
    f = data['flows']
    subpaths = data['subpaths'] 
    W = 1e5
    S = {str(min(map(int,V)))}
    D = {str(max(map(int,V)))}
    
    print()

    
    # import library
    from docplex.mp.model import Model

    # model name
    Flow = Model(name = 'Flow Decomposition')

    # define variables
    x = {(i,j,k): Flow.binary_var(name="x_{0}_{1}_{2}".format(i,j,k)) for (i,j) in E for k in range(0,K)}
    w = {(k): Flow.integer_var(name="w_{0}_".format(k)) for k in range(0,K)}
    z = {(i,j,k): Flow.integer_var(name="z_{0}_{1}_{2}".format(i,j,k)) for (i,j) in E for k in range(0,K)}
    r = {(k,s): Flow.binary_var(name="r_{0}_{1}".format(k,s)) for k in range(0,K) for s in range(0,len(subpaths))}

    # flow conservation
    for k in range(0,K):
        for i in V:
            if i in S:
                Flow.add_constraint(sum(x[i,j,k] for j in V if i!=j and(i,j) in E) - sum(x[j,i,k] for j in V if i!=j and (j,i) in E) == 1)  
            if i in D:
                Flow.add_constraint(sum(x[i,j,k] for j in V if i!=j and (i,j) in E) - sum(x[j,i,k] for j in V if i!=j and (j,i) in E) == -1)
            if i not in S and i not in D:
                Flow.add_constraint(sum(x[i,j,k] for j in V if i!=j and (i,j) in E) - sum(x[j,i,k] for j in V if i!=j and (j,i) in E) == 0)
    
    # flow balance
    for (i,j) in E:
        Flow.add_constraint(f[i,j] == sum(z[i,j,k] for k in range(0,K)))
        
    # linearization
    for (i,j) in E:
        for k in range(0,K):
            Flow.add_constraint(z[i,j,k] <= W*x[i,j,k])
            Flow.add_constraint(w[k] - (1 - x[i,j,k])*W <= z[i,j,k])
            Flow.add_constraint(z[i,j,k] <= w[k])
            
    # subpath constraints
    for k in range(0,K):
        for sp_len in range(0,len(subpaths)):
            subpath_edges = list(more_itertools.pairwise(subpaths[sp_len]))
            Flow.add_constraint(sum(x[i,j,k] for (i,j) in subpath_edges) >= len(subpath_edges)*r[k,sp_len])
    
    for sp_len in range(0,len(subpaths)):
        Flow.add_constraint(sum(r[k,sp_len] for k in range(0,K)) >=1)

    
    # objective function
    Flow.minimize(1)
    
    Flow.parameters.threads = 1
    Flow.parameters.mip.display = 0
    Flow.parameters.timelimit = 60

    sol = Flow.solve(log_output=False)
    print('status',Flow.solve_details.status)

    if Flow.solve_details.status == 'integer infeasible' or Flow.solve_details.status == 'time limit exceeded, no integer solution':
        data['messsage'] = "infeasible"
        return data

    elif Flow.solve_details.status == 'time limit exceeded':
        print('time limit')
  
    print('FD time',Flow.solve_details.time)

    # calculate averages
    x_sol = {(i,j,k):Flow.solution.get_value(x[(i,j,k)]) for (i,j) in E for k in range(0,K)}
    w_sol = {(k):Flow.solution.get_value(w[(k)]) for (k) in range(0,K)}
    paths =[list() for i in range(0,K)]
    for k in range(0,K):
        for (i,j) in E:
            if x_sol[i,j,k] == 1:
                paths[k].append((i,j))
               
    data['weights'] = w_sol
    data['solution'] = paths
    data['message'] = 'solved'
    data['runtime'] = Flow.solve_details.time
    
    return data;
    

In [4]:
def FD_Algorithm(data):
    
    listOfEdges = data['edges']
    solutionMap = data['graph']
    solutionSet = 0
    Kmin = data['minK']
    solutionWeights = 0

    for i in range(1,len(listOfEdges)+1):
        data = flowMultipleDecomposition(data,i)
        if data['message'] == "solved":
            solutionSet = data['solution']
            solutionWeights = data['weights']
            break;
    
    return data

In [5]:
def SolveInstances(Graphs,outfile,recfile):
    
    fp = open(outfile,'w+')
    fc = open(recfile,'w+')
    
    for i in range(0,1): 
        
        f = {}
        Edges = set()
        V = set()
        listOfEdges = Graphs[i]['list of edges']
        for k in range(0,len(listOfEdges)):
            (a,b,c) = (listOfEdges[k])
            Edges.add((a,b))
            V.add(a)
            V.add(b)
            f[a,b] = int(float(c))
            
        
        # creation of graphs
        G = nx.DiGraph()
        G.add_edges_from(Edges,weights = f)
        G.add_nodes_from(V)
        
        
        # calculating topological order
        G_top = nx.topological_sort(G)
        G_top_list = list(nx.topological_sort(G))
        topological_cuts = sorted(G.degree, key=lambda x: x[1], reverse=True)[0][1]
        
        # definition of data
        data = {'edges' : Edges,
                'flows' : f,
                'vertices' : V,
                'graph' : G,
                'subpaths': Graphs[i]['subpaths'],
                'Kmax' : len(Edges),
                'weights' : {},
                'message': {},
                'minK': topological_cuts,
                'runtime': 0,
        }
        
        data = FD_Algorithm(data)      
        
        w_sol = data['weights']
        paths = data['solution']
        print(len(data['weights']),data['runtime'],file=fp)
        for k in range(0,len(data['weights'])):
            print(w_sol[k],paths[k],file=fc)
        print("------",file=fc)
        
    
    return 0

In [6]:
path = "../Example/Subpath/"
text_files = [f for f in os.listdir(path) if f.endswith('.graph')]
outputfile = "results_cplex_subpath.txt"
recordfile = "results_cplex_subpath_details.txt"

for i in range(0,len(text_files)):    
    filename = text_files[i]
    data = read_input(''.join([path,filename]),4)
    data = SolveInstances(data,outputfile,recordfile)
    
print("Done")

{0: ['1', '3'], 1: ['1', '2'], 2: ['0', '1'], 3: ['2', '3']}

status integer infeasible

status integer infeasible

status integer optimal solution
FD time 0.007883071899414062
Done
